<H1>Лабораторна робота 2</H1>

In [3]:
#імпортуємо необхідні бібліотеки
import urllib.request as urllib2
import pandas as pd
from datetime import datetime, date
import os
import glob

Визначаємо допоміжні функції для завантаження даних

In [4]:
#повертає поточну дату у потріному форматі
def get_date():
    return date.today()

#повертає поточний час у потрібному форматі
def get_time():
    return datetime.now().strftime("%H-%M-%S")

#створює ім'я для завантаженого файлу
def get_file_name(region_number):
    current_date = get_date()
    current_time = get_time()
    return f'vhi_id_{region_number}_{current_date}_{current_time}.csv'

#видаляє з завантажених даних непотрібний текст
def preprocess_data(data):
    temp = str(data).split('\\n')[1:-1]
    temp[0] = temp[0][:-4]
    temp[1] = temp[1][9:]
    return '\n'.join(temp)

Тепер створюємо функцію для завантаження даних. Параметром є номер області

In [5]:
def download_data(region_number):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_number}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib2.urlopen(url)

    data = vhi_url.read()
    preprocessed_data = preprocess_data(data)

    file_name = get_file_name(region_number)
    if not os.path.exists('data'):
        os.mkdir('data')
    output_file = open(f'data/{file_name}', 'w')
    output_file.write(preprocessed_data)
    output_file.close()

Завантажимо дані для кожної з 27 областей

In [6]:
regions_number = 27
for region_number in range(1, regions_number + 1):
    download_data(region_number)

Визначаємо функцію для читання завантажених даних. Параметром є шлях до директорії з файлами

In [7]:
#допоміжна функція для додавання індексу регіона у фрейм
def get_region_idex(full_path, directory_path):
    fragment = full_path[len(directory_path) + 7:]
    return int(fragment[:fragment.index('_')])

def read_data(directory_path='./data/'):
    dfs = []
    for path in glob.glob(os.path.join(directory_path, '*.csv')):
        df = pd.read_csv(path, index_col=False)
        df['region_index'] = get_region_idex(path, directory_path)
        df.rename(columns={' SMN': 'SMN', ' VHI': 'VHI'}, inplace=True)
        df = df[df['VHI'] != -1]
        dfs.append(df)
    return pd.concat(dfs)

Зчитуємо дані

In [8]:
df = read_data()

Перевіримо результат

In [9]:
df.head()

,year,week,SMN,SMT,VCI,TCI,VHI,region_index
0,1982,1,0.059,258.24,51.11,48.78,49.95,10
1,1982,2,0.063,261.53,55.89,38.20,47.04,10
2,1982,3,0.063,263.45,57.30,32.69,44.99,10
3,1982,4,0.061,265.10,53.96,28.62,41.29,10
4,1982,5,0.058,266.42,46.87,28.57,37.72,10


Як бачимо, читання відбулось правильно

Додамо до фрейму назви областей

In [10]:
# об'єкт, де ключами є номера областей, а значеннями - їх назви
indexes_to_names = {
    1: 'Черкаська',
    2: 'Чернігівська',
    3: 'Чернівецька',
    4: 'Республіка Крим',
    5: 'Дніпропетровська',
    6: 'Донецька',
    7: 'Івано-Франківська',
    8: 'Харківська',
    9: 'Херсонська',
    10: 'Хмельницька',
    11: 'Київська',
    12: 'Місто Київ',
    13: 'Кіровоградська',
    14: 'Луганська',
    15: 'Львівська',
    16: 'Миколаївська',
    17: 'Одеська',
    18: 'Полтавська',
    19: 'Рівенська',
    20: 'Севастополь',
    21: 'Сумська',
    22: 'Тернопільська',
    23: 'Закарпатська',
    24: 'Вінницька',
    25: 'Волинська',
    26: 'Запорізька',
    27: 'Житомирська',
}

In [11]:
#функція, що додає назви областей
def add_region_names(df, names_obj):
    df['region_name'] = df['region_index'].map(lambda id: names_obj[id])

Застосувуємо створену функцію

In [12]:
add_region_names(df, indexes_to_names)

Перевіримо

In [13]:
df.head()

,year,week,SMN,SMT,VCI,TCI,VHI,region_index,region_name
0,1982,1,0.059,258.24,51.11,48.78,49.95,10,Хмельницька
1,1982,2,0.063,261.53,55.89,38.20,47.04,10,Хмельницька
2,1982,3,0.063,263.45,57.30,32.69,44.99,10,Хмельницька
3,1982,4,0.061,265.10,53.96,28.62,41.29,10,Хмельницька
4,1982,5,0.058,266.42,46.87,28.57,37.72,10,Хмельницька


Як бачимо, було додано стовпець region_name

Тепер реалізуємо процедури вибірок

In [14]:
#ряд VHI для заданої області за рік, якщо рік не задано, повертаються дані за всі роки
def vhi_series(df, region_name, year=None):
    result_df = df[df['region_name'] == region_name]
    if year:
        result_df = result_df[result_df['year'] == year]
    return result_df['VHI']

#максимум ряду VHI для заданої області за рік, якщо рік не задано, використовуються дані за всі роки
def max_vhi(df, region_name, year=None):
    return vhi_series(df, region_name, year).max()

#мінімум ряду VHI для заданої області за рік, якщо рік не задано, використовуються дані за всі роки
def min_vhi(df, region_name, year=None):
    return vhi_series(df, region_name, None).min()

Протестуємо роботу функцій

In [15]:
region_name = 'Хмельницька'
series = vhi_series(df, region_name)
print(f'ряд VHI для {region_name}:\n', series.values)

max = max_vhi(df, region_name)
print('максимум:', max)

min = min_vhi(df, region_name)
print('мінімум:', min)

ряд VHI для Хмельницька:
 [49.95 47.04 44.99 ... 49.04 51.52 55.04]
максимум: 79.4
мінімум: 18.41


Як бачимо, результати є правильними

Реалізуємо інші функції

In [16]:
#роки з екстремальними посухами для заданої області
def years_with_extreme_droughts(df, region_name):
    return df[(df['region_name'] == region_name) & (df['VHI'] < 15)]['year'].unique()

#роки з помірними посухами для заданої області
def years_with_moderate_droughts(df, region_name):
    return df[(df['region_name'] == region_name) & (df['VHI'] < 35) & (df['VHI'] > 15)]['year'].unique()

Перевіримо їх роботу

In [17]:
region_name = 'Місто Київ'
extreme = years_with_extreme_droughts(df, 'Місто Київ')
print(f'роки з екстремальними посухами у {region_name}:\n', extreme)

moderate = years_with_moderate_droughts(df, 'Місто Київ')
print(f'роки з помірними посухами у {region_name}:\n', moderate)

роки з екстремальними посухами у Місто Київ:
 [1999 2000]
роки з помірними посухами у Місто Київ:
 [1982 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996
 1999 2000 2001 2006 2008 2009 2010 2011 2012 2013 2015 2019 2020 2021
 2022 2023]


Як бачимо, результати є правильними